In [1]:
# pyscience imports
import os
import sys
import glob
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-darkgrid")
# plt.style.use("dark_background")
sns.set(style="ticks", context="talk")
# %matplotlib inline
# run for jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
from pprint import pprint
import re
import string
import numpy as np
import pandas as pd
import copy
import itertools
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from urllib import request
from collections import Counter
import json
from joblib import Parallel, delayed
import multiprocessing

import nltk
from nltk import word_tokenize
from nltk import bigrams
from nltk import trigrams
from nltk import ngrams
from nltk.stem import PorterStemmer
from nltk import word_tokenize

In [3]:
dbhome   = '/home/alal/Dropbox/0_GradSchool/1_HW/452/Hansard/'
ricehome = '/home/apoorval/HW/452/Hansard/'
home = ricehome
twfy_root = home + 'input/twfy/'
twfy = twfy_root + '2010+/'
jsons = home + 'input/twfy/jsons/'
%cd $home

/home/apoorval/HW/452/Hansard


# Parse XMLs 
Dry run

## Function to parse XML and populate list of speeches 

In [6]:
def twfy_parser(infile, inpdir = twfy, outdir = jsons):
    tree = ET.parse(inpdir + infile)
    root = tree.getroot()
    list_of_speeches = []
    for sp in tree.findall('speech'):
        # dict with metadata
        d =  sp.attrib
        # speech tag
        s = sp.findall('p')
        if s:
            # extract speeches
            statement = [x.text for x in s if x is not None]
            statement = ' '.join([x for x in statement if x])
        d['speech'] = statement
        list_of_speeches.append(d)
    # if there is a single speech
    if len(list_of_speeches) > 0:
        outfile = outdir + infile.replace('xml', 'json')
        json.dump(list_of_speeches, open(outfile, 'w'))
        return list_of_speeches

In [7]:
parl = os.listdir(twfy)

## Parallelise Parsing and Writing to JSON

In [8]:
%%time
num_cores = multiprocessing.cpu_count()
res = Parallel(n_jobs = num_cores)(delayed(twfy_parser)(p) for p in parl)

CPU times: user 4.93 s, sys: 981 ms, total: 5.91 s
Wall time: 25.5 s


In [9]:
len(res)

1730

In [10]:
speeches = [x for x in res if x is not None]
len(speeches)
big_speech_list = [speech for day in speeches for speech in day]

1569

In [11]:
len(big_speech_list)

434719

In [12]:
outfile = jsons + '_all_speeches.json'
json.dump(big_speech_list, open(outfile, 'w'))

# Read JSONs

In [13]:
big_speech_list = json.load(open(jsons + '_all_speeches.json'))

In [14]:
len(big_speech_list)

434719

In [15]:
big_speech_list[:5]

[{'id': 'uk.org.publicwhip/debate/2017-10-09a.74.1',
  'speakername': 'Chris Stephens',
  'person_id': 'uk.org.publicwhip/person/25306',
  'colnum': '74',
  'time': '18:19:00',
  'url': '',
  'speech': 'On a point of order, Mr Speaker. Last month, '},
 {'id': 'uk.org.publicwhip/debate/2011-10-17a.603.2',
  'nospeaker': 'true',
  'colnum': '603',
  'time': '14:30:00',
  'url': 'http://www.publications.parliament.uk/pa/cm201011/cmhansrd/cm111017/debtext/111017-0001.htm#111017-0001.htm_dpthd0',
  'speech': 'The Secretary of State was asked—'},
 {'id': 'uk.org.publicwhip/debate/2011-10-17a.603.4',
  'speakerid': 'uk.org.publicwhip/member/40395',
  'speakername': 'Annette Brooke',
  'oral-qnum': '1',
  'colnum': '603',
  'time': '14:30:00',
  'url': 'http://www.publications.parliament.uk/pa/cm201011/cmhansrd/cm111017/debtext/111017-0001.htm#1110177000414',
  'speech': 'What assessment he has made of the 2011 GCSE results for academies; and if he will make a statement.'},
 {'id': 'uk.org.pub

# Speaker Lists 

In [20]:
speakers = {}

In [24]:
for x in big_speech_list:
    speakerid = x.get("person_id", None)
    speakername = x.get("speakername", None)
    if speakername is not None:
        if x.get(speakerid, None) is not None:
            speakers[speakername].append(speakerid)
        else:
            speakers[speakername] = speakerid

In [25]:
{k: speakers[k] for k in list(speakers)[:5]}

{'Chris Stephens': None,
 'Annette Brooke': 'uk.org.publicwhip/member/40395',
 'Nick Gibb': None,
 'Chi Onwurah': 'uk.org.publicwhip/member/40419',
 'James Gray': None}

In [18]:
speaks = pd.DataFrame.from_dict(speakers, orient = 'index')
speaks.columns = ['person_name']

In [19]:
speaks.head()

,person_name
Chris Stephens,None
Annette Brooke,uk.org.publicwhip/member/40395
Nick Gibb,None
Chi Onwurah,uk.org.publicwhip/member/40419
James Gray,None


In [61]:
%cd $twfy_root
%mkdir tmp -p

/home/apoorval/HW/452/Hansard/input/twfy


In [62]:
speaks.to_csv(twfy_root + 'tmp/speaker_list.csv')